In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import json
import os
import logging
import requests

import functools as ft

from tqdm import tqdm_notebook

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

%load_ext autoreload
%autoreload 2

logging.basicConfig(level=logging.INFO)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
KUCHIKOMI_API = 'https://api.gnavi.co.jp/PhotoSearchAPI/v3/'

# APIの概要を確認

仕様は[こちら](https://api.gnavi.co.jp/api/manual/)

In [21]:
# 機密情報をうっかり流出させないように、別ファイルに保存してあるものを取得
with open('./../gurunabi_api.json','r') as file: 
    api_params=json.load(file)

In [22]:
kuchikomi_params = api_params.copy()
kuchikomi_params['range'] = 3
kuchikomi_params['latitude'] = '35.681236'
kuchikomi_params['longitude'] = '139.767125'

In [17]:
@ft.lru_cache()
def get_kuchikomi_datas():
    data = requests.get(
        KUCHIKOMI_API,
        params = kuchikomi_params
    ).json()
    
    return data

In [18]:
kuchikomi_data = get_kuchikomi_datas()

In [42]:
kuchikomi_data['response'].keys()

len(kuchikomi_data['response'])-3

15

In [43]:
def kuchikomi2df(kuchikomi_data):
    res = kuchikomi_data['response']
    commnts  = []
    num_comment = len(res) -3
    
    for i in range(num_comment):
        com_df = pd.io.json.json_normalize(res[str(i)])
        commnts.append(com_df)
    
    return pd.concat(commnts)
    

In [45]:
kuchikomi2df(kuchikomi_data).head()

,photo.areaname_l,photo.areaname_m,photo.areaname_s,photo.category,photo.comment,photo.distance,photo.image_url.url_1024,photo.image_url.url_200,photo.image_url.url_250,photo.image_url.url_320,photo.latitude,photo.longitude,photo.menu_finish_flag,photo.menu_id,photo.menu_name,photo.messages.first_shop_message.message_body,photo.messages.first_shop_message.send_date,photo.messages.shop_message_count,photo.messages.user_message_count,photo.nickname,photo.photo_genre_id,photo.photo_genre_name,photo.photo_scene_id,photo.photo_scene_name,photo.prefname,photo.shop_id,photo.shop_name,photo.shop_url,photo.total_score,photo.umaso_count,photo.update_date,photo.vote_id
0,東京駅・丸の内・日本橋,東京駅（八重洲）,東京駅（八重洲）,ステーキハンバーグ,すね肉のコリコリした食感が美味。,59,https://mr.gnavi.co.jp/cont/menu_image/88/53/1...,https://mr.gnavi.co.jp/cont/menu_image/88/53/1...,https://mr.gnavi.co.jp/cont/menu_image/88/53/1...,https://mr.gnavi.co.jp/cont/menu_image/88/53/1...,35.681596,139.767608,0,25622,すねバーグ,,,0,0,ぐるなび会員,1,料理・ドリンク,,,PREF13:東京都,1019663,ＳＴＥＡＫ ＆ ＨＡＭＢＵＲＧＶｉｍｏｎ,https://r.gnavi.co.jp/m1vmb2mw0000/?ak=7qF1fl1...,3.0,0,2011-01-25T19:43:46+09:00,169384
0,東京駅・丸の内・日本橋,東京駅（八重洲）,東京駅（八重洲）,ステーキハンバーグ,友人に連れられて伺いました。\r\nボリュームのある粗引き肉のハンバーグで、タルタルステーキ...,59,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,35.681596,139.767608,0,262317,Ｂランチ とろバーグ,,,0,0,puntopunto,1,料理・ドリンク,,,PREF13:東京都,1019663,ＳＴＥＡＫ ＆ ＨＡＭＢＵＲＧＶｉｍｏｎ,https://r.gnavi.co.jp/m1vmb2mw0000/?ak=7qF1fl1...,3.0,0,2011-10-05T22:14:31+09:00,356692
0,東京駅・丸の内・日本橋,東京駅（八重洲）,東京駅（八重洲）,ステーキハンバーグ,肉汁タップリのジューシーなバーグ。\r\n肉自体に味がシッカリ付いているので、そのまま食べて...,59,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,35.681596,139.767608,0,266176,Ａランチ すねバーグ,,,0,0,K_S,1,料理・ドリンク,,,PREF13:東京都,1019663,ＳＴＥＡＫ ＆ ＨＡＭＢＵＲＧＶｉｍｏｎ,https://r.gnavi.co.jp/m1vmb2mw0000/?ak=7qF1fl1...,3.0,0,2011-11-05T22:45:27+09:00,379070
0,東京駅・丸の内・日本橋,東京駅（八重洲）,東京駅（八重洲）,ステーキハンバーグ,ハンバーグの挽肉が荒引きで美味しいです。ステーキを頂いてるみたい。,59,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,35.681596,139.767608,0,262317,Ｂランチ とろバーグ,,,0,0,kyon1120,1,料理・ドリンク,,,PREF13:東京都,1019663,ＳＴＥＡＫ ＆ ＨＡＭＢＵＲＧＶｉｍｏｎ,https://r.gnavi.co.jp/m1vmb2mw0000/?ak=7qF1fl1...,5.0,0,2011-11-15T20:06:14+09:00,398502
0,東京駅・丸の内・日本橋,東京駅（八重洲）,東京駅（八重洲）,ステーキハンバーグ,しっかりとお肉の味がするハンバーグ！付け合わせの野菜もおいしい！,59,https://mr.gnavi.co.jp/cont/menu_image/88/53/4...,https://mr.gnavi.co.jp/cont/menu_image/88/53/4...,https://mr.gnavi.co.jp/cont/menu_image/88/53/4...,https://mr.gnavi.co.jp/cont/menu_image/88/53/4...,35.681596,139.767608,0,266176,Ａランチ すねバーグ,,,0,0,mamikko,1,料理・ドリンク,,,PREF13:東京都,1019663,ＳＴＥＡＫ ＆ ＨＡＭＢＵＲＧＶｉｍｏｎ,https://r.gnavi.co.jp/m1vmb2mw0000/?ak=7qF1fl1...,5.0,0,2011-11-29T23:15:50+09:00,430506
